In [1]:
!pip install torch transformers accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.4 MB/s eta 0:00:00


In [2]:
!pip install datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 13.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "microsoft/Phi-3-mini-4k-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["qkv_proj", "o_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,718,592 || all params: 3,825,798,144 || trainable%: 0.1233


In [21]:
data = []

for _, row in df.iterrows():
    example = {
        "instruction": "Generate cooking directions using the given recipe title and ingredients.",
        "input": (
            f"Recipe title: {row['recipe_title']}\n"
            f"Ingredients: {row['ingredient_text']}"
        ),
        "output": row['directions_text']
    }
    data.append(example)

# Save to JSON
with open("phi3_recipes_2000.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Saved {len(data)} recipes to phi3_recipes_2000.json")

Saved 2000 recipes to phi3_recipes_2000.json


In [17]:
import pandas as pd
import json

# Load CSV
df = pd.read_csv("/content/recipes_extended.csv", engine='python', on_bad_lines='skip')

# Take only first 2000 recipes
df = df.head(2000)

In [22]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/phi3_recipes_2000.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [51]:
def format_example(example):
    raw_input = example["input"]
    directions = example["output"]

    # Default fallbacks
    title = ""
    ingredients = ""

    # Parse title and ingredients from the input string
    lines = raw_input.splitlines()
    for line in lines:
        if line.startswith("Recipe title:"):
            title = line[len("Recipe title:"):].strip()
        elif line.startswith("Ingredients:"):
            ingredients = line[len("Ingredients:"):].strip()

    # If parsing fails for some weird row, fall back
    if not title:
        title = "Unknown recipe"
    if not ingredients:
        ingredients = "Not specified"

    # Build a proper chat-style prompt & answer
    user_content = (
        "Write a full, well-formatted cooking recipe using the given title and ingredients.\n\n"
        f"Title: {title}\n"
        f"Ingredients:\n{ingredients}"
    )

    assistant_content = (
        f"Recipe: {title}\n\n"
        f"Ingredients:\n{ingredients}\n\n"
        f"Instructions:\n{directions}"
    )

    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    return {"text": text}

dataset = dataset.map(format_example)
print(dataset["train"][0]["text"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<|user|>
Write a full, well-formatted cooking recipe using the given title and ingredients.

Title: Air Fryer Potato Slices with Dipping Sauce
Ingredients:
ketchup beer worcestershire sauce onion powder cayenne baking potatoes olive oil cooking spray garlic powder salt freshly ground black pepper<|end|>
<|assistant|>
Recipe: Air Fryer Potato Slices with Dipping Sauce

Ingredients:
ketchup beer worcestershire sauce onion powder cayenne baking potatoes olive oil cooking spray garlic powder salt freshly ground black pepper

Instructions:
combine ketchup, beer, worcestershire sauce, onion powder, and cayenne in a small saucepan. bring to a boil, then reduce heat, and simmer for 3 to 5 minutes. remove from heat, and cool. cover and store in the refrigerator until ready to use. preheat the air fryer to 400 degrees f (200 degrees c). spray the basket with cooking spray or line with a disposable parchment liner. slice potatoes 1/4-inch thick (use a mandoline if you have one), and place in a bo

In [30]:
!pip install -U accelerate

In [52]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./phi3-recipes",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=False,
    bf16=False,
    optim="paged_adamw_32bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    processing_class=tokenizer,
    dataset_text_field="text",
)

trainer.train()

Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Step,Training Loss


KeyboardInterrupt: 

In [41]:
import os

os.environ["ACCELERATE_MIXED_PRECISION"] = "no"
os.environ["WANDB_DISABLED"] = "true"

In [43]:
trainer.save_model("phi3-recipes-lora")

In [49]:
messages = [
    {"role": "user", "content": """Recipe title: Air Fryer Peanut Chicken. Ingredients: ["2 tablespoons Thai sweet chili sauce", "1 tablespoon soy sauce", "1 tablespoon Sriracha", "1 lime, zested and juiced", "1 teaspoon granulated garlic", "1 tablespoon fresh ginger", "1/3 cup creamy peanut butter", "salt to taste", "6 skinless, boneless chicken thighs", "cooking spray", "2 cups cooked rice, or as needed", "chopped cilantro", "dry-roasted peanuts"]"""}
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

output = model.generate(
    inputs,
    max_new_tokens=200,
    temperature=0.6
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Recipe title: Air Fryer Peanut Chicken. Ingredients: ["2 tablespoons Thai sweet chili sauce", "1 tablespoon soy sauce", "1 tablespoon Sriracha", "1 lime, zested and juiced", "1 teaspoon granulated garlic", "1 tablespoon fresh ginger", "1/3 cup creamy peanut butter", "salt to taste", "6 skinless, boneless chicken thighs", "cooking spray", "2 cups cooked rice, or as needed", "chopped cilantro", "dry-roasted peanuts"] preheat an air fryer to 400 degrees f (200 degrees c). combine thai sweet chili sauce, soy sauce, sriracha, lime zest and juice, granulated garlic, and ginger in a bowl. stir in peanut butter and salt until smooth. add chicken thighs and toss to coat. spray the air fryer basket with cooking spray. arrange chicken in an even layer in the basket. cook, turning once halfway through, until chicken is no longer pink at the center and juices run clear, about 15 minutes. an instant-read thermometer inserted into the center should read 165 degrees f (74 degrees c). meanwhile, stir r